<a href="https://colab.research.google.com/github/ronish645/Ronish380/blob/main/9_01_Lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
data=pd.read_csv('adult.csv')

Int his lab assignment, you have two tasks:
1. Create a dataset from the `adult.csv` where the new dataset will have columns with random number of missing values. The ratio of missing values should randomly change from 5% to 10%. The numerical columns will have `np.NaN`, and categorical ones will have `?` as missing values.
For example,the first column, age, is numerical. It may have a randomly chosen 6% of the age values as missing.
At this stage, take they type of each coumn as the datatype. For example, `marital_status` looks like a numerical column at the beginning. Tka it as a numerical column and replace a random percentage of values with `np.NaN`.
2. Create a pipeline with transformers and a classifier `SVC(kernel='rbf', C=1, gamma='scale')`. When creating the transformers, make sure that each column is correctly classified as numerical or categorical. For example, `marital_status` may have a `float` data type, but in reality it is a categorical variable. For that reason, you need to update the data types for all categorical variables.Report the accuracy for your model.
3. Repeat the second step for the original dataset without any missing values. Compatre your result with that from the second step.

In [3]:
data.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education_num',
       'marital_status', 'occupation', 'relationship', 'race', 'sex',
       'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
       'income'],
      dtype='object')

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             48842 non-null  float64
 1   workclass       48842 non-null  int64  
 2   fnlwgt          48842 non-null  int64  
 3   education       48842 non-null  int64  
 4   education_num   48842 non-null  float64
 5   marital_status  48842 non-null  int64  
 6   occupation      48842 non-null  int64  
 7   relationship    48842 non-null  int64  
 8   race            48842 non-null  int64  
 9   sex             48842 non-null  int64  
 10  capital_gain    48842 non-null  float64
 11  capital_loss    48842 non-null  float64
 12  hours_per_week  48842 non-null  float64
 13  native_country  48842 non-null  int64  
 14  income          48842 non-null  object 
dtypes: float64(5), int64(9), object(1)
memory usage: 5.6+ MB


In [ ]:
# Create a copy of the dataset to modify
modified_data = data.copy()

In [ ]:
# Iterate over each column in the dataset
for column in modified_data.columns:
    # Generate a random number between 5 and 10
    missing_ratio = np.random.uniform(0.05, 0.1)
     # Determine the data type of the column
    data_type = modified_data[column].dtype

    # Check if the column is numerical or categorical
    if data_type == 'int64' or data_type == 'float64':
        # Generate a mask with True for missing values
        mask = np.random.choice([True, False], size=len(modified_data), p=[missing_ratio, 1-missing_ratio])

        # Replace the values with NaN based on the mask
        modified_data.loc[mask, column] = np.nan
    else:
        # Generate a mask with True for missing values
        mask = np.random.choice([True, False], size=len(modified_data), p=[missing_ratio, 1-missing_ratio])

        # Replace the values with "?" based on the mask
        modified_data.loc[mask, column] = "?"


In [ ]:
print(modified_data)

            age  workclass    fnlwgt  education  education_num  \
0      0.030671        7.0   77516.0        9.0       1.134739   
1      0.837109        6.0       NaN        9.0       1.134739   
2     -0.042642        4.0  215646.0       11.0      -0.420060   
3      1.057047        4.0  234721.0        1.0            NaN   
4           NaN        4.0  338409.0        9.0       1.134739   
...         ...        ...       ...        ...            ...   
48837  0.030671        4.0  215419.0        9.0       1.134739   
48838       NaN        0.0       NaN        NaN      -0.420060   
48839 -0.042642        4.0  374983.0        9.0       1.134739   
48840  0.397233        4.0   83891.0        9.0            NaN   
48841 -0.262580        5.0  182148.0        NaN            NaN   

       marital_status  occupation  relationship  race  sex  capital_gain  \
0                 4.0         1.0           1.0   4.0  1.0      0.148453   
1                 2.0         4.0           0.0   4.0  

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [ ]:
# Split the modified_data into features and target
X = modified_data.drop('income', axis=1)
y = modified_data['income']

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Define the transformer for numerical columns
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])


In [ ]:
# Define the transformer for categorical columns
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

In [ ]:
preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_transformer, X.select_dtypes(include=['int64', 'float64']).columns),
    ('cat', categorical_transformer, X.select_dtypes(include=['object']).columns)
])

In [ ]:
# Create the pipeline with transformers and classifier
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', SVC(kernel='rbf', C=1, gamma='scale'))
])

In [ ]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['age', 'workclass', 'fnlwgt', 'education', 'education_num',
       'marital_status', 'occupation', 'relationship', 'race', 'sex',
       'capital_gain', 'capital_loss', 'hours_per_week', 'native_country'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index([], dtype='object'))])),
                ('classifier', SVC(C=1))])

In [ ]:
# Make predictions on the testing data
y_pred = pipeline.predict(X_test)

In [ ]:
# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)

In [ ]:
# Split the data into features and target
X_original = data.drop('income', axis=1)
y_original = data['income']


In [ ]:
# Split the data into training and testing sets
X_train_original, X_test_original, y_train_original, y_test_original = train_test_split(X_original, y_original, test_size=0.2, random_state=42)

In [ ]:
# Fit the pipeline on the training data
pipeline.fit(X_train_original, y_train_original)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['age', 'workclass', 'fnlwgt', 'education', 'education_num',
       'marital_status', 'occupation', 'relationship', 'race', 'sex',
       'capital_gain', 'capital_loss', 'hours_per_week', 'native_country'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index([], dtype='object'))])),
                ('classifier', SVC(C=1))])

In [ ]:
# Make predictions on the testing data
y_pred_original = pipeline.predict(X_test_original)

In [ ]:
# Calculate the accuracy of the model
accuracy_original = accuracy_score(y_test_original, y_pred_original)

In [ ]:
# Compare the accuracies
print("Accuracy with missing values:", accuracy)
print("Accuracy without missing values:", accuracy_original)

Accuracy with missing values: 0.5259494318763436
Accuracy without missing values: 0.5611628621148531
